In [1]:
%%capture --no-stderr
!pip install composio-langgraph==0.7.15 langgraph langchain_openai

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["COMPOSIO_API_KEY"] = userdata.get('COMPOSIO_API_KEY')

In [33]:
from typing import Literal
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode


# composio = Composio()
# user_id = "pg-test-f211b19e-0250-449c-a"  # Use your email or a unique identifier


In [17]:
!composio add googlecalendar


> Adding integration: Googlecalendar...

Select auth mode:  (OAUTH2, BEARER_TOKEN): ac_IL-OumkbcuB3
Error: 'ac_IL-OumkbcuB3' is not one of 'OAUTH2', 'BEARER_TOKEN'.
Select auth mode:  (OAUTH2, BEARER_TOKEN): OAUTH2
Please authenticate googlecalendar in the browser and come back here. URL: https://backend.composio.dev/api/v3/s/SjBOEAeg
⚠ Waiting for googlecalendar authentication...
✔ googlecalendar added successfully with ID: 5b3fa5f8-6957-4750-a242-a70e51627319


In [18]:
!composio add gmail



> Adding integration: Gmail...

Select auth mode:  (OAUTH2, BEARER_TOKEN): OAUTH2
Please authenticate gmail in the browser and come back here. URL: https://backend.composio.dev/api/v3/s/PVeLIQFi
⚠ Waiting for gmail authentication...
✔ gmail added successfully with ID: ce3e2415-63b9-499a-8efc-90e03d46df95


In [34]:
from composio_langgraph import Action, ComposioToolSet, App

composio_toolset = ComposioToolSet()

In [35]:
# Get the tools (replace user_id etc as appropriate)

tools = composio_toolset.get_tools(
    actions=[
        Action.GOOGLECALENDAR_FIND_FREE_SLOTS,
        Action.GOOGLECALENDAR_CREATE_EVENT,
        Action.GMAIL_CREATE_EMAIL_DRAFT
    ]
)

In [36]:
tools

[StructuredTool(name='GMAIL_CREATE_EMAIL_DRAFT', description='Creates a gmail email draft, supporting to/cc/bcc, subject, plain/html body (ensure `is html=true` for html), attachments, and threading.', args_schema=<class 'composio.utils.shared.CreateEmailDraftRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x7dd205f4b740>),
 StructuredTool(name='GOOGLECALENDAR_CREATE_EVENT', description='Creates an event on a google calendar, needing rfc3339 utc start/end times (end after start) and write access to the calendar. by default, adds the organizer as an attendee unless exclude organizer is set to true.', args_schema=<class 'composio.utils.shared.CreateEventRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x7dd2ee059300>),
 StructuredTool(name='GOOGLECALENDAR_FIND_FREE_SLOTS', description='Finds free/busy time slots in google calen

In [37]:
tool_node = ToolNode(tools)

In [38]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [39]:
model_with_tools = model.bind_tools(tools)

In [40]:
model_with_tools.kwargs

{'tools': [{'type': 'function',
   'function': {'name': 'GMAIL_CREATE_EMAIL_DRAFT',
    'description': 'Creates a gmail email draft, supporting to/cc/bcc, subject, plain/html body (ensure `is html=true` for html), attachments, and threading.',
    'parameters': {'properties': {'attachment': {'default': None,
       'description': 'File to attach to the email.',
       'type': 'string'},
      'bcc': {'default': [],
       'description': "'Bcc' (blind carbon copy) recipient email addresses.",
       'items': {'type': 'string'},
       'type': 'array'},
      'body': {'description': 'Email body content (plain text or HTML); `is_html` must be True if HTML. Please provide a value of type string. This parameter is required.',
       'type': 'string'},
      'cc': {'default': [],
       'description': "'Cc' (carbon copy) recipient email addresses.",
       'items': {'type': 'string'},
       'type': 'array'},
      'extra_recipients': {'default': [],
       'description': "Additional 'To' re

In [41]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

In [42]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [43]:
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# Step 9: Compile workflow
app = workflow.compile()

In [44]:
for chunk in app.stream(
    {
        "messages": [
            ("human",
             "Find a 30-minute free slot tomorrow afternoon, create a Google Meet event called 'Project Sync with John Doe' for November 9th,2025, and draft a Gmail invitation to John at johndoe@example.com.")
        ]
    },
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Find a 30-minute free slot tomorrow afternoon, create a Google Meet event called 'Project Sync with John Doe' for November 9th,2025, and draft a Gmail invitation to John at johndoe@example.com.
================================== Ai Message ==================================
Tool Calls:
  GOOGLECALENDAR_FIND_FREE_SLOTS (call_Lv5eueVwVO5yuQdzfbsSdM0V)
 Call ID: call_Lv5eueVwVO5yuQdzfbsSdM0V
  Args:
    time_min: 2025-11-09T12:00:00Z
    time_max: 2025-11-09T17:00:00Z
    timezone: UTC
================================= Tool Message =================================
Name: GOOGLECALENDAR_FIND_FREE_SLOTS

{"data": {"calendars": {"primary": {"busy": [], "free": [{"end": "2025-11-09T17:00:00+00:00", "start": "2025-11-09T12:00:00+00:00"}]}}, "kind": "calendar#freeBusy", "timeMax": "2025-11-09T17:00:00.000Z", "timeMin": "2025-11-09T12:00:00.000Z"}, "error": null, "successfull": true, "successful": true, "logId": "l